In [ ]:
import numpy as np
import pandas as pd
import string
import re
import regex

import nltk
from nltk import SnowballStemmer
snowball = SnowballStemmer(language="russian")
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.compose import ColumnTransformer

#nltk.download('stopwords')
#nltk.download('punkt')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
stop_words = list([word for word in stopwords.words('russian') if not word in ['без','не','нет']])
stop_words=set(stop_words)
stopword = stop_words.union({'недел', 'недель', 'беремен', 'витамин','тип', 'феврал','февраль','степен', 'посл', 'ст', 'ог', 'оа', 'мед', 'тел', 'год', 'брак', 'зарегистрирова', 'ммртст', 'уд', 'мин', 'мм', 'рт', 'мкг', 'мг', 'кг', 'ад', 'сутк', 'мес', 'род', 'вес', 'рожден', 'гр', 'январ', 'октябр','январь', 'октябрь', 'мат', 'мать', 'отец', 'бабушк', 'бабушка','мам', 'мама','сестр', 'дедушк','сестра', 'дедушка',  'брат', 'гб', 'отц', 'пап', 'дед', 'дяд','дядя', 'ма', 'папин', 'дедшк', 'племянник', 'бабаушк', 'сын', 'доч','дочь', 'бабк', 'декабрь', 'март','апрел','апрель','май','июн','июл','июнь','июль','август','сентябр','октябр','ноябр','сентябрь','октябрь','ноябрь','декабр', 'нед', 'годас', 'ден'})

In [ ]:
def clean_tweet(tweet):
    tweet = str(tweet)
    tweet = regex.sub(r'\p{P}(?!\d)',' ',tweet) #все символы пунктуации
    tweet = re.sub('\d+', ' ', tweet)#цифры
    tweet = re.sub('\+', ' ', tweet)#специальные символы
    tweet = re.sub('\r', ' ', tweet)#специальные символы
    tweet = re.sub('\n', ' ', tweet)#специальные символы
    tweet = re.sub('\s\S\s', ' ', tweet) #один символ
    tweet = regex.sub(r'\s+',' ', tweet) #увеличенное количество пробельных символов
    return tweet 

def clean(x):
    x = x.split(' ')
    x = [morph.parse(k)[0].normal_form for k in x] #Лемматизация
    x = [snowball.stem(k) for k in x]
    x = ' '.join([k for k in x if k not in stopword]) #удаление стоп-слов
    return x

def data_preprocessing(x): 
    #выравнивание регистра
    x = str(x).lower()
    x = clean_tweet(x)
    x = clean(x)
    return x

def clean_tweet1(tweet):
    tweet = str(tweet)
    tweet = regex.sub(r'\p{P}(?!\d)',' ',tweet) #все символы пунктуации
    tweet = re.sub('\)', ' ', tweet)#специальные символы
    tweet = re.sub('\s\d+\s', ' ', tweet)#цифры
    tweet = re.sub('\+', ' ', tweet)#специальные символы
    tweet = re.sub('\r', ' ', tweet)#специальные символы
    tweet = re.sub('\n', ' ', tweet)#специальные символы
    tweet = re.sub('\s\S\s', ' ', tweet) #один символ
    tweet = regex.sub(r'\s+',' ', tweet) #все пробельные символы
    return tweet 

def data_preprocessing1(x): 
    #выравнивание регистра
    x = str(x).lower()
    x = clean_tweet1(x)
    x = clean(x)
    return x

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False))
        
def token(df, name):
    tokenizer = RegexpTokenizer(r'\w+')
    df0=df.copy(deep=True)
    for i in range (len(df0[name])):
        c= (tokenizer.tokenize(df0[name][i]))
        for j in range (len(c)):
            if c[j]=='не':
                if j+1<len(c):
                    c[j]='не_'+c[j+1]
                    c[j+1]=''
                else:
                    c[j]=''
            elif c[j]=='без':
                if j+1<len(c):
                    c[j]='без_'+c[j+1]
                    c[j+1]=''
                else:
                    c[j]=''
            if c[j]=='отрицательн':
                if j!=0:
                    c[j]=c[j-1]+'_отрицательн'
                    c[j-1]=''   
        df0[name][i]=' '.join([k for k in c if k != ''])
    return(df0)
        
def norm(df, name):
    s=['не_быт', 'отрица', 'нет', 'не_отягот', 'без_особен','не_имет', 'вредн', 'вред', 'привычк', 'превычек', 'проф вредност отриц','проф вредност отр', 'ипроф вредност отриц', 'не_имет', 'вред привычк отриц', 'не_кур', 'не_пит']
    tokenizer = RegexpTokenizer(r'\w+')
    df0=df.copy(deep=True)
    for i in range (len(df0[name])):
        c= (tokenizer.tokenize(df0[name][i]))
        for j in range (len(c)):
            if c[j] in s:
                c[j]=''
        df0[name][i]=' '.join([k for k in c if k != ''])
    return(df0)

In [ ]:
df = pd.read_csv('data/1_type.csv', delimiter=',') #считывание записей 
del df['Unnamed: 0']
df = df.dropna(axis='index', how='any', subset=['diagnos'])
df = df.fillna('')
df=df.reset_index(drop=True)
df.rename(columns = {'course ':'course'}, inplace = True)
df=df.drop_duplicates(keep='first')

df = df.loc[df['weeks'] > 0] #очистка от данные с подготовкой к беременности
df=df.reset_index(drop=True)

In [ ]:
#Препроцессинг данных

for name in ['value', 'diagnos', 'com','ill', 'heredity', 'habits', 'gynecology', 'inspection', 'husband', 'course', 'pills', 'previous','fetus']:
    df[name] = df[name].apply(data_preprocessing) #очистка от пукнтуации, стоп-слов, лемматизация
    df = token(df,name) #токенизация
    df = norm(df,name)
df['recommendations'] = df['recommendations'].apply(data_preprocessing1)
df = token(df,'recommendations')
df = norm(df,'recommendations')    

In [ ]:
#Определение триместра

df['trim']=0
for i in range(len(df)):
    if df['weeks'][i] < 14:
        df['trim'][i]=1
    if ((df['weeks'][i] >= 14)& (df['weeks'][i] <= 27)):
        df['trim'][i]=2
    if df['weeks'][i] > 27:
        df['trim'][i]=3
for i in range(len(df)):
    if df['age'][i] < 18:
        df['age'][i]=1
    if ((df['age'][i] >= 18)& (df['age'][i] <= 40)):
        df['age'][i]=2
    if df['age'][i] > 40:
        df['age'][i]=3

In [ ]:
#Группировка призноков

df['fir']=df['value']+' '+df['gynecology']+' '+df['inspection']+' '+df['fetus']+' '+df['course'] 
df['sec']=df['heredity']+' '+df['husband']+' '+df['habits'] 
df['thr']=df['previous']+' '+df['recommendations']+' '+df['pills']+' '+df['ill']
del df['husband']
del df['course']
del df['previous']
del df['fetus']
del df['value']
del df['ill']
del df['heredity']
del df['habits']
del df['gynecology']
del df['inspection']
del df['pills']
del df['action_id']
del df['id']
del df['recommendations']

In [ ]:
#Сохранение очищенных данных

df.to_csv('data/1.csv')